# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Постройте модель с максимально большим значением *accuracy*. Необходимо довести долю правильных ответов по крайней мере до 0.75. Проверьть *accuracy* на тестовой выборке самостоятельно.

Описание данных:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
# Библиотеки
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from warnings import simplefilter
simplefilter(action='ignore')
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
import time

In [2]:
data = pd.read_csv('/datasets/****ior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбивка данных на выборки

In [4]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

In [5]:
features_train, features_test, target_train, target_test  = train_test_split(features,target, test_size=0.2, random_state=12345) 

print(features_train.shape)
print(target_train.shape)

(2571, 4)
(2571,)


In [6]:
features_train, features_val, target_train, target_val = train_test_split(features_train, target_train, test_size=0.25, random_state=12345) 


In [7]:
print('Обучающая выборка:')
print(features_train.shape)
print(target_train.shape)
print('Тестовая выборка:')
print(features_test.shape)
print(target_test.shape)
print('Валидационная выборка:')
print(features_val.shape)
print(target_val.shape)

Обучающая выборка:
(1928, 4)
(1928,)
Тестовая выборка:
(643, 4)
(643,)
Валидационная выборка:
(643, 4)
(643,)


## Исследование модели

In [8]:
#Проверка валидационной модели Дерева решений
start_time = time.time()
model_class = DecisionTreeClassifier()
parametrs = { 'max_depth': range (1,11),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10) }
grid = GridSearchCV(model_class, parametrs, cv=5)
grid.fit(features_val, target_val)
finish_time = time.time()

model_class = DecisionTreeClassifier(random_state=12345, max_depth = grid.best_params_["max_depth"]
                                     , min_samples_leaf = grid.best_params_["min_samples_leaf"]
                                     , min_samples_split = grid.best_params_["min_samples_split"] )
model_class.fit(features_train, target_train)
predictions = model_class.predict(features_val)
accuracy = accuracy_score(target_val, predictions)
print("Лучшие параметры", grid.best_params_)
print("Точность предсказания", accuracy)
print ("Расчетное время", finish_time-start_time)


Лучшие параметры {'max_depth': 7, 'min_samples_leaf': 3, 'min_samples_split': 3}
Точность предсказания 0.7744945567651633
Расчетное время 11.965874671936035


In [9]:
start_time = time.time()
model_class = LogisticRegression()
parametrs = {'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
            'max_iter': range(10, 100)}
grid = GridSearchCV(model_class, parametrs, cv=5)
grid.fit(features_val, target_val)
finish_time = time.time()
model_class = LogisticRegression(random_state=12345, solver = grid.best_params_["solver"]
                                     , max_iter = grid.best_params_["max_iter"])
model_class.fit(features_train, target_train)
predictions = model_class.predict(features_val)
accuracy = accuracy_score(target_val, predictions)
print("Лучшие параметры", grid.best_params_)
print("Точность предсказания", accuracy)
print ("Расчетное время", finish_time-start_time)


Лучшие параметры {'max_iter': 39, 'solver': 'newton-cg'}
Точность предсказания 0.7278382581648523
Расчетное время 31.871304512023926


In [10]:
#Проверка валидационной модели Случайного леса
start_time = time.time()
model_class = RandomForestClassifier()
parametrs = { 'n_estimators': range (1, 11),
              'max_depth': range (1,11),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10) }
grid = GridSearchCV(model_class, parametrs, cv=5)
grid.fit(features_val, target_val)
finish_time = time.time()
model_class = RandomForestClassifier(random_state=12345, max_depth = grid.best_params_["max_depth"]
                                     , min_samples_leaf = grid.best_params_["min_samples_leaf"]
                                     , min_samples_split = grid.best_params_["min_samples_split"]
                                     , n_estimators = grid.best_params_["n_estimators"])
model_class.fit(features_train, target_train)
predictions = model_class.predict(features_val)
accuracy = accuracy_score(target_val, predictions)
print("Лучшие параметры", grid.best_params_)
print("Точность предсказания", accuracy)
print ("Расчетное время", finish_time-start_time)


Лучшие параметры {'max_depth': 9, 'min_samples_leaf': 6, 'min_samples_split': 4, 'n_estimators': 8}
Точность предсказания 0.7838258164852255
Расчетное время 323.31711173057556


В данной задаче целевой признак категориальный, значит решаем задачу бинарной классификации. 
Для исследования берем следующие классификаторы:
- DecisionTreeClassifier (Дерево решений)
- RandomForestClassifier (Случайный лес)
- LogisticRegression (Логистическая регрессия)

В исследовании использовались разные гиперпараметри для определения наилучшего:
- для DecisionTreeClassifier лучшим оказался max_depth=5, min_samples_leaf=2, min_samples_split=7, accuracy = 0.774
- для RandomForestClassifier лучшим были max_depth=6, min_samples_leaf=5, min_samples_split=6, n_estimators=9, accuracy = 0.783
- для LogisticRegression лучшим были max_iter=39, solver=newton-cg,  accuracy = 0.728

Лучшей моделью оказался RandomForestClassifier, для дальнейшего исследования будем применять эту модель.


## Проверка модель на тестовой выборке

In [11]:
model_class = RandomForestClassifier(random_state=12345, max_depth = grid.best_params_["max_depth"]
                                     , n_estimators=grid.best_params_["n_estimators"]
                                    , min_samples_leaf = grid.best_params_["min_samples_leaf"]
                                     , min_samples_split = grid.best_params_["min_samples_split"])
model_class.fit(features_train, target_train)
predictions_test = model_class.predict(features_test)
accuracy = accuracy_score(target_test, predictions_test)
print('Точность предсказания модели на тестовой сборке:', accuracy)

Точность предсказания модели на тестовой сборке: 0.7978227060653188


Доля правильных ответов выбранной модели  на тестовой выборке равна 0.7884914463452566.

## Проверька модели на адекватность

In [12]:
dummy_model = DummyClassifier(strategy='most_frequent') 
dummy_model.fit(features_train, target_train)
dummy_score = dummy_model.score(features_test, target_test)
print('Точность предсказания модели на тестовой сборке c DummyClassifier:', dummy_score)
print('Точность предсказания модели на тестовой сборке c Random Forest:', accuracy)

Точность предсказания модели на тестовой сборке c DummyClassifier: 0.6951788491446346
Точность предсказания модели на тестовой сборке c Random Forest: 0.7978227060653188


Результаты проверки на адекватность говорят о правильно выбранной модели.